## Instruções
- Redes Neuronais + Backpropagation

Neste trabalho você irá implementar uma rede neuronal com três camadas:

1. Camada de entrada: cada unidade representa uma dimensão do dado de entrada.

2. Camada oculta: cada unidade representa uma transformação a partir das unidades de entrada.

3. Camada de saída: cada unidade representa a chance da saída correspondente ser a correta.

Você irá utilizar a função Sigmóide para obter não-linearidade. Além disso, a função de perda a ser minimizada é a seguinte:

![title](formula.jpg)

onde m é a quantidade de entradas no treino, K é o número de saídas possíveis,  representa a saída correta de cada classe k em cada entrada (i), e similarmente representa a saída dada pela rede neuronal.

O dado a ser utilizado está anexado. Trata-se de 5000 entradas, onde cada entrada refere-se a um dígito escrito manualmente (i.e., MNIST dataset). Dessa forma, m=5000 e K=10. Cada entrada é dada por uma matriz de dimensões 28 por 28, ou seja, um vetor de 784 dimensões. A primeira coluna do arquivo sempre é o rótulo do dígito correto.

A rede neuronal a ser implementada deverá ter 784 unidades de entrada e 10 unidades de saída. Em seus experimentos, você deverá variar o número de unidades na camada oculta (25, 50, 100).

Além disso, você deverá comparar os seguintes algoritmos de cálculo de gradiente:

1. Gradient Descent: o gradiente é calculado após cada época (após as 5000 entradas serem processadas).

2. Stochastic Gradient Descent: o gradiente é calculado após cada entrada.

3. Mini-Batch: o gradiente é calculado após um certo número de entradas (considere 10 e 50).

Por fim, você também deverá variar a taxa de aprendizado: 0.5, 1, 10.

O documento a ser entregue deverá apresentar o resultado de seus experimentos. Ou seja, deverá apresentar discussão da variação do número de unidades na camada oculta para cada um dos três algoritmos de cálculo de gradiente. Você deverá apresentar gráficos mostrando a convergência do erro empírico para cada situação (unidades na camada oculta, algoritmo de cálculo do gradiente, taxa de aprendizado). Você deverá deixar claras todas as hipóteses que julgar serem pertinentes.

In [12]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras import optimizers
import numpy
import keras

# fix random seed for reproducibility
numpy.random.seed(7)

## Carregando Arquivo
Carregando o arquivo com os 5000 digitos e fazendo uma divisão de folds para fazer o cross-fold

In [13]:
# load pima indians dataset
dataset = numpy.loadtxt("/home/lfmendes/data/mestrado/machine-learning/ml-neural/data_tp1.csv", delimiter=",")

# split into input (X) and output (Y) variables
X = dataset[:,1:]
y = dataset[:,0]

y_cat = np_utils.to_categorical(y)
num_classes = y_cat.shape[1]

num_pixels = 784

print("Número de entradas: %s" % (len(y)))
print("Número de features: %s" % (len(X[0])))
print("Número de classes: %s" % (num_classes))

Número de entradas: 5000
Número de features: 784
Número de classes: 10


Criando um divisão de 5 folds stratified, isto é, mantendo o número de classes por fold

In [14]:
from sklearn.model_selection import StratifiedKFold

fold_size = 5
skf = StratifiedKFold(n_splits=fold_size, random_state=42,shuffle=True)

## Código para criação do modelo dado os parâmetros que podem ser variados

In [ ]:
def create_model(learning_rate=0.5,neuronios_ocultos=25):
    # create model   
    model = Sequential()
    model.add(Dense(num_pixels, activation='relu', input_shape=(784,)))
    model.add(Dense(neuronios_ocultos, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    # Compile model
    sgd = optimizers.SGD(lr=learning_rate)
    model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])    
   
    return model

## Definindo parâmetros de teste

In [16]:
batch_sizes=[1,10,50,5000]
#batch_sizes=[100,5000]
learning_rates=[0.5, 1, 10]
neuronios=[25,50,100]

## Fazendo experimentação
Dividindo em treino e teste usando 5 kfolds.
Para cada um dos parâmetros, será feito testes nos 5 folds e o resultado será a média da métrica escolhida

In [17]:
def experimentation(batch_size=100,learning_rate=0.5,neuronio=100,epochs=20, verbose=0):    
    fold = 0
    avg_error = 0.0
    avg_acc = 0.0
    first = True
    
    for train_index, test_index in skf.split(X, y):
        print("Fazendo teste nos fold " + str(fold))

        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_train = X_train.reshape(len(train_index), 784)
        X_test = X_test.reshape(len(test_index), 784)
        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255
        print(X_train.shape[0], 'train samples')
        print(X_test.shape[0], 'test samples')

        y_train = np_utils.to_categorical(y_train)
        y_test = np_utils.to_categorical(y_test)   

        model=create_model(learning_rate=learning_rate,neuronios_ocultos=neuronio)
        
        if(first):
            model.summary()
            first = False

        # Fit the model
        model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size, verbose=verbose)

        # Final evaluation of the model
        scores = model.evaluate(X_test, y_test, verbose=1)
        print("Baseline Error: %.2f%%" % (100-scores[1]*100))
        print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

        avg_acc = avg_acc + scores[1]*100

        fold=fold+1

    print(avg_acc/fold_size)    
    return (avg_acc/fold_size)

## Experimentando com batch size

In [ ]:
batches_results = {}
for batch in batch_sizes:
    print('Experimentando com batch size igual a ' + str(batch))
    batches_results[str(batch)] = experimentation(batch_size=batch)    

In [ ]:
import matplotlib.pyplot as plt

plt.bar(list(batches_results.keys()), batches_results.values(), color='g')
plt.show()

## Experimentando com learning_rates

In [20]:
learning_rates_results = {}
for learning_rate in learning_rates:
    print('Experimentando com learning_rates igual a ' + str(learning_rate))
    learning_rates_results[str(learning_rate)] = experimentation(learning_rate=learning_rate)    

plt.bar(list(learning_rates_results.keys()), learning_rates_results.values(), color='g')
plt.show()    

Experimentando com learning_rates igual a 0.5
Fazendo teste nos fold 0
3997 train samples
1003 test samples
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_64 (Dense)             (None, 784)               615440    
_________________________________________________________________
dense_65 (Dense)             (None, 100)               78500     
_________________________________________________________________
dense_66 (Dense)             (None, 10)                1010      
Total params: 694,950
Trainable params: 694,950
Non-trainable params: 0
_________________________________________________________________
1003/1003 [==============================] - 0s 84us/step
Baseline Error: 5.58%

acc: 94.42%
Fazendo teste nos fold 1
3999 train samples
1001 test samples
1001/1001 [==============================] - 0s 78us/step
Baseline Error: 5.59%

acc: 94.41%
Fazendo teste nos fold 2
4001 train samples


KeyboardInterrupt: 

## Experimentando com neuronios

In [ ]:
neuronios_results = {}
for neuronio in neuronios:
    print('Experimentando com learning_rates igual a ' + str(neuronio))
    neuronios_results[str(neuronio)] = experimentation(neuronio=neuronio)    

plt.bar(list(neuronios_results.keys()), neuronios_results.values(), color='g')
plt.show()  